In [2]:
import requests
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [ ]:
# =========================================
# Crypto Risk Thermometer - Unified Notebook
# =========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------------------
# 1. Data Layer - Load or Fetch Data
# -----------------------------------------

# For now, assume you have already fetched and stored CSVs:
# - macro_cpi.csv, macro_rates.csv, sp500.csv
# - btc_mcap.csv, total_mcap.csv, btc_dominance.csv, btc_hashrate.csv

# Example (replace with your own paths/APIs later):
cpi = pd.read_csv("dashboard/cpi_data.csv", parse_dates=["date"], index_col="date")
rates = pd.read_csv("dashboard/federal_funds_rate_data.csv", parse_dates=["date"], index_col="date")
sp500 = pd.read_csv("data/sp500.csv", parse_dates=["date"], index_col="date")
btc_mcap = pd.read_csv("data/btc_mcap.csv", parse_dates=["date"], index_col="date")
total_mcap = pd.read_csv("dashboard/global_mcap.csv", parse_dates=["date"], index_col="date")
btc_dom = pd.read_csv("dashboard/btc_dominance_monthly.csv", parse_dates=["date"], index_col="date")
hashrate = pd.read_csv("data/btc_hashrate.csv", parse_dates=["date"], index_col="date")

# Resample monthly
datasets = [cpi, rates, sp500, btc_mcap, total_mcap, btc_dom, hashrate]
for df in datasets:
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
    df = df.resample("M").mean()

# -----------------------------------------
# 2. Transformation Layer - Scoring Functions
# -----------------------------------------

def normalize_series(series, invert=False):
    """Normalize series to 0–100 scale. Optionally invert (if higher = worse)."""
    s = (series - series.min()) / (series.max() - series.min()) * 100
    if invert:
        s = 100 - s
    return s

# MacroScore
def compute_macro_score(cpi, rates, sp500):
    cpi_score = normalize_series(cpi["value"], invert=True)   # lower CPI = better
    rate_score = normalize_series(rates["value"], invert=True)  # lower rates = better
    sp_score = normalize_series(sp500["value"], invert=False)   # higher S&P = better
    return (cpi_score + rate_score + sp_score) / 3

# CryptoFlowScore
def compute_crypto_flow_score(btc_dom, total_mcap):
    dom_score = normalize_series(btc_dom["value"], invert=True)  # falling dom = risk-on
    mcap_score = normalize_series(total_mcap["value"], invert=False)
    return (dom_score + mcap_score) / 2

# MinerHealth
def compute_miner_health(hashrate):
    return normalize_series(hashrate["value"], invert=False)  # rising/stable = better

# -----------------------------------------
# 3. Aggregation Layer - Final Thermometer
# -----------------------------------------

macro_score = compute_macro_score(cpi, rates, sp500)
flow_score = compute_crypto_flow_score(btc_dom, total_mcap)
miner_score = compute_miner_health(hashrate)

thermometer = 0.4*macro_score + 0.4*flow_score + 0.2*miner_score

# Combine into DataFrame
df_scores = pd.DataFrame({
    "MacroScore": macro_score,
    "FlowScore": flow_score,
    "MinerHealth": miner_score,
    "Thermometer": thermometer
})

# -----------------------------------------
# 4. Visualization Layer
# -----------------------------------------

plt.figure(figsize=(12,6))
df_scores["Thermometer"].plot(label="Risk Thermometer", color="black")
plt.axhspan(0,30, color="red", alpha=0.2, label="Defensive")
plt.axhspan(30,60, color="orange", alpha=0.2, label="Mixed")
plt.axhspan(60,100, color="green", alpha=0.2, label="Risk-On")
plt.title("Crypto Risk Thermometer (0–100)")
plt.ylabel("Score")
plt.legend()
plt.show()

# Sub-scores
df_scores[["MacroScore", "FlowScore", "MinerHealth"]].plot(figsize=(12,6))
plt.title("Thermometer Components")
plt.show()


yello
